In [1]:
import pandas as pd
import spacy
import re


In [2]:
nlp = spacy.load("en_core_web_sm")
df = pd.read_csv("data/dataset_resume/Resume/Resume.csv",chunksize=100)

In [3]:
keywords = [
    "experience", "project", "management", "responsibility",
    "work history", "work experience", "job description",
    "role", "tasks", "positions", "certification", "abilities",
    "technical skills", "summary", "profile", "accomplishments"
]

In [4]:
skills_domain = [
    "accounting", "client relations", "data analysis",
    "customer service", "marketing", "statistics",
    "project management", "financial analysis",
    "communication", "problem-solving",
    "programming", "python", "java", "c++",
    "machine learning", "nlp", "tensorflow",
    "pytorch", "analytic skills", "leadership",
    "teamwork", "collaboration", "debugging",
    "testing", "agile", "git", "sql",
    "databases", "networking", "cloud computing",
    "aws", "azure", "project coordination",
    "risk management", "decision making",
    "critical thinking", "conflict resolution",
    "time management", "creative thinking",
    "research", "report writing", "self-motivated",
    "self-starter"
]

In [5]:
def clean_html(text):
    text = re.sub(r"<[^>]+>", " ", str(text))
    return re.sub(r"\s+", " ", text).strip()



In [6]:
def extract_relevant_sentences(text):
    if pd.isna(text): return []
    text = clean_html(text)
    doc = nlp(text)

    relevant_sentences = []

    for sent in doc.sents:
        sentence_text = sent.text.strip()
        sentence_lower = sentence_text.lower()

        for keyword in keywords:
            if keyword in sentence_lower:
                relevant_sentences.append(sentence_text)
                break;
    return relevant_sentences



In [7]:
def match_skills(text):
    matched= []
    text_lower = str(text).lower()
    for skill in skills_domain:
        skill_lower = skill.lower()
        if skill_lower in text_lower:
            matched.append(skill)
    return matched

In [15]:
for chunk in df:
    print(f"This is loop:${chunk}")
    chunk["Relevant Sentences"] = chunk["Resume_html"].apply(extract_relevant_sentences)
    matched_skills_list = []

    for row in chunk["Relevant Sentences"]:
        sentences = " ".join(row)
        matched_skills  = match_skills(sentences)
        matched_skills_list.append(matched_skills)

    chunk["matched_skills"] = pd.Series(matched_skills_list).values
    chunk["raw_data"] = chunk.apply(lambda row:(row["Relevant Sentences"],row["matched_skills"]),axis=1)

chunk["raw_data"].to_csv("Resume_processed.csv",mode='a',index=False,header=False)



# After your for loop:
new_df = pd.read_csv("Resume_processed.csv", header=None, names=["Relevant Sentences", "Matched Skills"])

print(new_df.iloc[1])  # Prints the 2nd row
print(df_processed["Relevant Sentences"])

    

Relevant Sentences    (['AIR TRAFFIC CONTROLLER Summary Retired Air ...
Matched Skills                                                      NaN
Name: 1, dtype: object


NameError: name 'df_processed' is not defined

In [13]:
df["Relevant_sentences"]

TypeError: 'TextFileReader' object is not subscriptable